In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
%matplotlib inline

#                                                       Outline

## Section 1: Decision Trees
#### Section 1.1 Intuition Underlying Tree Based Models
#### Section 1.2 Applying Tree Based Models
#### Section 1.3 Shortcomings & Limitations

## Section 2: Ensemble Methods & Random Forests
#### Section 2.1 Intuition underlying ensemble approaches
#### Section 2.2 Toy Classification Example
#### Section 2.3 Time series Regression Example
#### Section 2.4 MNIST Dataset Classification

# Section 1.1 Intuition Underlying Tree Based Models
### I Spy With My Little Eye...
There is a popular game played by children, during road trips, called "I Spy With My Little Eye". It involves one person choosing something they saw and the other has to infer said something based on a series of yes/no questions.
![alt text](Fig1.pdf "I Spy With My Little Eye")

During this game, the player partitions the space of total possible objects, with each question defining a new split, till there's just one member left in the accessible space.

For instance, the first question takes away the inanimate objects in the space.
![alt text](Fig2b.pdf "I Spy With My Little Eye")

The second question removes the 4-legged friends from the picture.
![alt text](Fig2c.pdf "I Spy With My Little Eye")

The final question removes the terrestrial creatures, in favor of our feathered friends.
![alt text](Fig2d.pdf "I Spy With My Little Eye")

In mathematical parlance, the little girl is solving a multi-class classification problem using disjoint partitions on the space of measurable events using directed acyclic graphs. Used thus, these DAGs are Classification And Regression Trees.
![alt text](Fig3a.pdf "I Spy With My Little Eye")

Classification & Regression Trees (CARTs) are non-parametric models, that learn such "splits" on the feature space from data, to define a mapping from different partitions of the feature space to values on the target space.
![alt text](Fig4.pdf "I Spy With My Little Eye")

# Section 1.2 Applying Tree Based Models
In this section, we apply tree based models to datasets and analyze their performance to visualize their strangths and weaknesses. But first, we start off by generating some data for classification.

In [ ]:
X, y = make_blobs(n_samples=300, centers=4,random_state=0, cluster_std=0.75)
plt.figure(figsize=(7,7))
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='gist_rainbow')
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X, y)

In the last code cell, first we instantiate the decision tree model ("tree = DecisionTreeClassifier(max_depth=3)"). 
The second step is fitting the model basde on the data. The process of fitting involves evaluating between a series of splits of the feature space and choosing the best at each step via a greedy approach.

A tree defines a partition on the space, and every node of the tree defines a split of the dataset. The tree is a sequence of splits, chosen based on the data. At every step, possible splits are evaluated based on increase in homogeneity of data due to the split.

![alt text](Fig5.pdf "I Spy With My Little Eye")

There are various arguments that can be passed to the classifier during instantiation. The primary ones include:

a) max_depth=n,  This determines the number of splits in the tree. 

b) criterion="gini" or "entropy", This determines the criterion used to evaluate the split. 
etc.


We shall experiment over the maximum depth criterion, later. For the moment, let's keep that fixed at 3

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='gist_rainbow',clim=(y.min(), y.max()), zorder=3)
xx, yy = np.meshgrid(np.linspace(np.min(X[:,0]),np.max(X[:,0]), num=200),np.linspace(np.min(X[:,1]),np.max(X[:,1]), num=200))
Z = tree.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
n_classes = len(np.unique(y))
contours = plt.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap='gist_rainbow', zorder=1)
plt.savefig('Fig6f.pdf', bbox_inches='tight')

In this step, we vary the maximum depth of the tree from 3 to 6 to 10. As we see in the ensuing figures, the tree's predictions improve a tad, but then quickly devolve to overfitting.
![alt text](Fig6a3.pdf "I Spy With My Little Eye")
![alt text](Fig6c5.pdf "I Spy With My Little Eye")
![alt text](Fig6e10.pdf "I Spy With My Little Eye")

Over-fitting turns out to be a property of decision trees. It is very easy to go too deep in the tree, and thus to fit details of the particular data rather than the overall properties of the distributions they are drawn from. 
![alt text](Fig7.pdf "I Spy With My Little Eye")


# Section 2. Random Forests
In this scenario, we use Ensemble Methods and use a randomized set of such "trees" to create a "forest". The basis of ensemble methods lies in the famous Condorcet’s Jury Theorem: A group wants to arrive at the “correct” decision via majority vote, wherein each individual has a probability p of voting for the correct decision. What should the size of the group be for optimal performance?
It has been shown that if the individual is doing better than random guessing, then the more independent adjudicators in the group, the better the chances of getting the "right" answer.

Intuitive example: The performance of the “Ask The Audience” lifeline in Who Wants To Be A Millionaire? 

(92% accuracy for “Ask The Audience” vs only 65% for “Phone A Friend”)

In this manner, Aggregating randomized models decreases the variance of the ensemble, while still retaining the low bias.

## Section 2.1 Random Forests: Toy Classification Example

In [ ]:
model = RandomForestClassifier(n_estimators=500, random_state=0, max_depth=5)

In [ ]:
model.fit(X,y)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap='gist_rainbow',clim=(y.min(), y.max()), zorder=3)
xx, yy = np.meshgrid(np.linspace(np.min(X[:,0]),np.max(X[:,0]), num=200),np.linspace(np.min(X[:,1]),np.max(X[:,1]), num=200))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
n_classes = len(np.unique(y))
contours = plt.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap='gist_rainbow', zorder=1)

#### Exercise:
In this exercise, we will be applying random forest classifier model on a cancer dataset. The set consists of measurements of 569 patients, with features describing the tumors, and the target classifying the tumors as malignant or benign. 

We import this dataset, split it into training and testing sets, and train a random forest classifier on this set.

In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target
X.shape, y.shape

In [ ]:
data.feature_names

In [ ]:
data.target_names

In [ ]:
#Step 1: Split the data into train and test sets (80%-20%)


In [ ]:
# Step 2: Instantiate a random forest classifier model with 100 estimators and a random state of 42. 
# If you complete this exercise early, Feel free to test the max_depth=4/5/6... hyperparameter too.


In [ ]:
# Step 3: Fit the model on the training data, 
#and evaluate its score on the training and the test data (model.score(X_test, y_test))


#### Deriving Insight from the Random Forest Model

In [ ]:
breast_cancer_features = [x for i,x in enumerate(data.feature_names)]
 
def breast_cancer_feature_importances_plot(model):
    plt.figure(figsize=(15,8))
    n_features = 30
    plt.barh(range(n_features), clf.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), breast_cancer_features)
    plt.title('Random Forest Classifier: Feature Importances')
    plt.xlabel("Feature Importance")
    plt.ylabel("Feature Name")
    plt.ylim(-1, n_features)
 
breast_cancer_feature_importances_plot(clf)
plt.show()

## Section 2.2 Random Forests: Time series Regression Example
In the previous section we considered random forests within the context of classification. Random forests can also be made to work in the case of regression (that is, continuous rather than categorical variables). The estimator to use for this is the RandomForestRegressor, and the syntax is very similar to what we saw earlier.

In [ ]:
rng = np.random.RandomState(42)
x = 10 * rng.rand(200)

def modelts(x, sigma=0.3):
    fast_oscillation = np.sin(5 * x)
    slow_oscillation = np.sin(0.5 * x)
    noise = sigma * rng.randn(len(x))
    return slow_oscillation + fast_oscillation + noise

y = modelts(x)
plt.scatter(x, y)

In [ ]:
forest = RandomForestRegressor(200)
forest.fit(x[:, None], y)

xfit = np.linspace(0, 10, 1000)
yfit = forest.predict(xfit[:, None])
ytrue = modelts(xfit, sigma=0)

plt.scatter(x, y,alpha=0.5)
plt.plot(xfit, yfit, '-r');
plt.plot(xfit, ytrue, '-k', alpha=0.5);

The true model is shown in the smooth gray curve, while the random forest model is shown by the jagged red curve. As you can see, the non-parametric random forest model is flexible enough to fit the multi-period data, without us needing to specifying a multi-period model.

## Section 2.3 Random Forests: MNIST Dataset Classification
As a final example, we apply the Random Forest Algorithm to the (too) well known MNIST dataset, for classifying images

In [ ]:
digits = load_digits()
fig = plt.figure(figsize=(3, 3)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(25):
    ax = fig.add_subplot(5, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    ax.text(0, 4, str(digits.target[i]))

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,random_state=0)
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

### Exercise solutions

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8 ,random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
print("Accuracy on training data: {:.2f}".format(clf.score(X_train, y_train)))
print("Accuracy on test data: {:.2f}".format(clf.score(X_test, y_test)))